In [1]:
from google.colab import files

uploaded = files.upload()

Saving colleges_train (1).csv to colleges_train (1).csv


In [2]:
from google.colab import files

uploaded = files.upload()

Saving colleges_test_features (1).csv to colleges_test_features (1).csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Load the data
df_train = pd.read_csv('colleges_train (1).csv')
df_test = pd.read_csv('colleges_test_features (1).csv')

# Define the features and target
features = ['adm_rate', 'satv25', 'satv50', 'satv75', 'satm25', 'satm50', 'satm75',
            'pell_grant_rate', 'fed_loan_rate', 'ug', 'ug_men', 'ug_women', 'ug_white',
            'ug_black', 'ug_hispanic', 'ug_asian', 'ug_25plus', 'first_gen',
            'faculty_salary', 'ft_faculty_rate', 'math_deg', 'engi_deg', 'bio_deg',
            'sci_deg', 'endowment', 'booksupply', 'roomboard']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
df_train[features] = imputer.fit_transform(df_train[features])
df_test[features] = imputer.transform(df_test[features])

# Scale features
scaler = StandardScaler()
df_train[features] = scaler.fit_transform(df_train[features])
df_test[features] = scaler.transform(df_test[features])

# Split the training data
X_train, X_val, y_train, y_val = train_test_split(df_train[features], df_train['tuition'], test_size=0.2, random_state=42)

# Train Lasso model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

# Predict on validation set
y_pred = lasso.predict(X_val)

# Calculate MSE
mse = mean_squared_error(y_val, y_pred)

# Predict tuition on the test set
predicted_tuition = lasso.predict(df_test[features])

# Calculate residuals from the training set
residuals = y_train - lasso.predict(X_train)

# Estimate the standard deviation of the residuals
std_residual = np.std(residuals)

# Calculate the 10% and 90% quantiles for each predicted tuition assuming a normal distribution centered at each predicted value
df_test['Predicted_Tuition'] = predicted_tuition
df_test['Lower_Quantile_Tuition'] = predicted_tuition - 1.645 * std_residual  # 10% quantile of normal distribution
df_test['Upper_Quantile_Tuition'] = predicted_tuition + 1.645 * std_residual  # 90% quantile of normal distribution

# Save the three columns to a new CSV file
df_quantiles = df_test[['Predicted_Tuition', 'Lower_Quantile_Tuition', 'Upper_Quantile_Tuition']]
df_quantiles.to_csv('predicted_tuition_quantiles_individual.csv', index=False)
print('Individual quantile CSV file created: predicted_tuition_quantiles_individual.csv')

# Calculate the mean validation error from the Lasso model
mean_validation_error = np.mean((y_val - y_pred) ** 2)

# Extract the coefficients of each attribute used in the Lasso model
coefficients = pd.Series(lasso.coef_, index=features)

print('Mean Validation Error:', mean_validation_error)
print('Coefficients of each attribute:')
print(coefficients)

Individual quantile CSV file created: predicted_tuition_quantiles_individual.csv
Mean Validation Error: 37624758.4593253
Coefficients of each attribute:
adm_rate          -1231.614087
satv25             1245.294080
satv50              278.463240
satv75             -888.534861
satm25             -812.391306
satm50             2113.129771
satm75             1652.546530
pell_grant_rate   -5983.413308
fed_loan_rate      4476.423736
ug                -2420.895801
ug_men             -621.573615
ug_women              0.000000
ug_white          -1841.327453
ug_black             52.970321
ug_hispanic        1539.212163
ug_asian            560.478957
ug_25plus          -167.027035
first_gen          -564.415497
faculty_salary     1365.582021
ft_faculty_rate     953.981978
math_deg            150.686244
engi_deg           -102.479099
bio_deg             173.399989
sci_deg             747.047057
endowment           999.295388
booksupply           14.983119
roomboard          1646.357314
dtype: flo

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.544e+09, tolerance: 4.359e+06
  model = cd_fast.enet_coordinate_descent(
